In [294]:
import requests

In [295]:
url = "https://www.worldometers.info/coronavirus/country/spain/"

In [296]:
req = requests.get(url)

In [297]:
req

<Response [200]>

In [298]:
from lxml.html import fromstring

In [299]:
root = fromstring(req.text)

In [300]:
root

<Element html at 0x7efbe828a0b0>

In [301]:
scripts = root.xpath(".//script")

In [302]:
for script in scripts:
    if script.text_content().strip().__contains__("Highcharts.chart"):
        value = script.text_content().strip()
        break

In [303]:
import re

In [304]:
lines = re.compile(r'[\n\t]')

In [305]:
value = lines.sub('', value)

In [306]:
comments = re.compile(r'\/\*(.*?)\*\/')

In [307]:
value = comments.sub('', value)

In [308]:
spaces = re.compile(r'[ ]{2,}')

In [309]:
value = spaces.sub(' ', value)

In [310]:
value = value.replace('\'', '\"')

In [311]:
chart = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )(.*?)(\)\;)')

In [312]:
value = chart.search(value).group(0)

In [313]:
chart_title = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )')

In [314]:
title = chart_title.search(value).group(0).replace('Highcharts.chart(\"', '').replace('\", ', '')

In [315]:
title

'total-currently-infected-linear'

In [316]:
value = chart_title.sub('', value)

In [317]:
chart_end = re.compile(r'\)\;')

In [318]:
value = chart_end.sub('', value)

In [319]:
options = re.compile(r'[\w]+\:')

In [320]:
words = options.findall(value)

In [321]:
words = list(set(words))

In [322]:
for word in words:
    value = value.replace(word, '\"' + word.replace(':', '') + '\":')

In [323]:
chart_properties = re.compile(r'(\"chart\"\:)(.*?)(\}\,)')

In [324]:
value = chart_properties.sub('', value)

In [325]:
import json

In [326]:
value = json.loads(value)

In [327]:
value.keys()

dict_keys(['title', 'xAxis', 'yAxis', 'legend', 'credits', 'plotOptions', 'series', 'responsive'])

In [340]:
column = value['yAxis']['title']['text']

In [341]:
y = [datetime.strptime(val + ', 2020', '%b %d, %Y') for val in value['xAxis']['categories']]

In [342]:
x = value['series'][0]['data']

In [343]:
import pandas as pd

In [351]:
data = pd.DataFrame()

In [352]:
data['Date'] = y

In [353]:
data[column] = x

In [354]:
data.set_index('Date', inplace=True)

In [357]:
data.to_csv(title)